In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 8.8MB/s 
     |████████████████████████████████| 3.0MB 24.4MB/s 
     |████████████████████████████████| 890kB 54.1MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=56772b85ca2f697a26e1a19302ed078f9a777cc592888220fc6bcb5735343f4f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaConfig
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from transformers import AdamW, RobertaForSequenceClassification

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive/
/gdrive


In [ ]:
import pandas as pd
df = pd.read_csv('My Drive/Colab_Notebooks/selected_foods.csv')
df.head()

,sentence,label
0,i love this water it does not taste good unles...,4
1,we have been looking for pesto that tastes as ...,4
2,i make own sausages back in ireland and this i...,4
3,anything that says enriched in the ingredients...,1
4,amazon normally does a fantastic job getting p...,2


In [ ]:
df.label = df.label-1
np.unique(df.label)

array([0, 1, 2, 3, 4])

In [ ]:
max_length = 80
learning_rate = 2e-5
batch_size = 64
epochs = 4

In [ ]:
# Create sentence and label lists
initial_sentences = df.sentence.values
initial_labels = df.label.values

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in initial_sentences]
# print ("Tokenize the first sentence:")
# print (tokenized_texts[0])

index = []
for i,x in enumerate(tokenized_texts):
    if len(x)<max_length:
        index.append(i)
sentences = []
labels=[]
for i in index:
   sentences.append(initial_sentences[i]) 
   labels.append(initial_labels[i])

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
tokenized_texts = [["[CLS]"] + tokenizer.tokenize(sent)  + ["[SEP]"] for sent in sentences]
# print ("Tokenize the first sentence:")
# print (tokenized_texts[0])

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=max_length, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)
model.cuda()

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters,
                     lr=learning_rate,
                     )


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
train_loss_set = []

for _ in trange(epochs, desc="Epoch"):
  # Training
  
  model.train()

  training_loss = 0
  nbatch_training_examples, nbatch_training_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    batch_input_ids, batch_input_mask, batch_labels = batch
    optimizer.zero_grad()
    loss = model(batch_input_ids,  attention_mask=batch_input_mask, labels=batch_labels)
    train_loss_set.append(loss[0])    

    # # Backward pass
    loss[0].backward()
    optimizer.step()
    
    
    # Update tracking variables
    training_loss =training_loss+ loss[0]
    nbatch_training_examples =nbatch_training_examples+ batch_input_ids.size(0)
    nbatch_training_steps =nbatch_training_steps+ 1

  print("Train loss: {}".format(training_loss/nbatch_training_steps))
    
    
  # Validation
  model.eval()

  # Tracking variables 
  evaluation_loss, evaluation_accuracy = 0, 0
  nbatch_evaluation_steps, nbatch_evaluation_examples = 0, 0
  eval_f1, eval_precision = 0,0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    batch_input_ids, batch_input_mask, batch_labels = batch
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(batch_input_ids, attention_mask=batch_input_mask)[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = batch_labels.to('cpu').numpy()

    tmp_evaluation_accuracy = accuracy_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten()) 
    eval_f1 += f1_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten(), average='weighted') 
    eval_precision += precision_score(np.argmax(logits, axis=1).flatten(), label_ids.flatten(), average='weighted')
    
    evaluation_accuracy =evaluation_accuracy+ tmp_evaluation_accuracy
    nbatch_evaluation_steps =nbatch_evaluation_steps+ 1

  print("Validation Accuracy: {}".format(evaluation_accuracy/nbatch_evaluation_steps))
  print("Validation F1 score: {}".format(eval_f1/nbatch_evaluation_steps))
  print("Validation precision: {}".format(eval_precision/nbatch_evaluation_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 1.0440702438354492


Epoch:  25%|██▌       | 1/4 [05:56<17:48, 356.30s/it]

Validation Accuracy: 0.6057351532567049
Validation F1 score: 0.6024972685130568
Validation precision: 0.6423119439664474
Train loss: 0.8549461364746094


Epoch:  50%|█████     | 2/4 [11:54<11:53, 356.72s/it]

Validation Accuracy: 0.6352490421455939
Validation F1 score: 0.632184682083606
Validation precision: 0.6466270113415857
Train loss: 0.7595056891441345


Epoch:  75%|███████▌  | 3/4 [17:51<05:56, 356.97s/it]

Validation Accuracy: 0.6465038314176245
Validation F1 score: 0.6512125010484131
Validation precision: 0.6797583795783413
Train loss: 0.6638429164886475


Epoch: 100%|██████████| 4/4 [23:48<00:00, 357.19s/it]

Validation Accuracy: 0.636566091954023
Validation F1 score: 0.631376061457401
Validation precision: 0.6577799743498751
